### Earthquake intensity zone statistics


In [128]:
### Method ###

##Output:
# Provinces with population per earthquake risk column. 
# 5 earthquake risk columns
# Shp or spatial database table

##Input:
# Province polygons
# 5 earthquake risk columns
# Shp or spatial database table

##Input:
# Province polygons
# Earthquake polyons
# Population raster

## Steps:

# 1.1
# Clip province polygons by earthquake polygons
# Explode polygons (multipart to single part)

# 2.1
# Extract population for each polygon using zonal statistics

# 2.2
# Extract building count for each polygon using spatial join and count

# 2.3
# Extract area for each polygon

# 3.1
# Summarize by both province, earthquake, building, area and intensity using pandas pivot table

# 4.1
# Join pivot table columns to original province polygons x3 (seperate for each dataset)

# 5.1
# QA

In [129]:
import os
import json
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats
from shapely.ops import transform
from datetime import datetime

import matplotlib.pyplot as plt

### Setting connection and parameters

In [130]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [131]:
pd.set_option('display.max_columns', None)

In [132]:
# Define projection
repro_crs = '+proj=cea'

### Data

In [133]:
pcode = 'adm1_pcode' ### OBS: Adjust based on admin level

#pop = r'D:\iMMAP\data\Afghanistan\Population\WorldPop\afg_worldpop_2020_UNadj_unconstrained.tif'
#pop = r'C:\Users\VMO\Desktop\afg_worldpop_2020_UNadj_unconstrained_proj4326.tif'

pop = r'D:\iMMAP\data\Afghanistan\afg_worldpop_2020_UNadj_unconstrained_projCEA.tif' #afg_worldpop_2020_UNadj_unconstrained_projCEA
adm = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm1_testclip2', con).to_crs(repro_crs)
#adm = gpd.read_file(r"D:\iMMAP\data\Afghanistan\HSDC-Official\afg_admbnda_adm1.shp")#.to_crs(repro_crs)
eq = gpd.read_file(r"D:\iMMAP\data\Afghanistan\HSDC-Official\afg_eq_hzda.shp").to_crs(repro_crs)

print('    Loading buildings   Start: {}'.format(datetime.now().strftime("%H:%M:%S")))
build = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft_centroids_testclip1_tiny', con).to_crs(repro_crs)
print('    Loading buildings   End  : {}'.format(datetime.now().strftime("%H:%M:%S")))

    Loading buildings   Start: 09:57:41
    Loading buildings   End  : 09:57:42


In [134]:
adm.crs

<Derived Projected CRS: +proj=cea +type=crs>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Cylindrical Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [135]:
eq.crs

<Derived Projected CRS: +proj=cea +type=crs>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Cylindrical Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### 1.1 Clip province polygons by earthquake polygons


In [136]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Use the overlay function to clip admin polygons by earthquake zones
overlay = gpd.overlay(adm, eq, how='intersection', keep_geom_type=None, make_valid=True)

# Post-process
adm_eq = overlay.explode()
adm_eq = adm_eq.to_crs(repro_crs)
adm_eq = adm_eq.reset_index()
adm_eq = adm_eq.drop(columns=['level_1', 'level_0'])

09:57:42


C:\Users\VMO\AppData\Local\Temp\ipykernel_16460\161707685.py:6: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  adm_eq = overlay.explode()


### 2.1 Extract population for each polygon using zonal statistics

In [137]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Produce list of dictionaries, each dictionary representing the sum for one polygon
zonalSt = rasterstats.zonal_stats(adm_eq, pop, stats = 'sum')

09:57:42


In [138]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Convert list of dictionaries to pandas dataframe (containing one column with each value)
df = pd.DataFrame(zonalSt)
df = df.rename(columns={'sum': 'pop_eq'})

09:57:44


In [139]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Add the values to the original table
df_concat = pd.concat([df, adm_eq], axis=1)

09:57:44


In [140]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Convert table back into a geodataframe    
gdf = gpd.GeoDataFrame(df_concat, geometry=df_concat.geometry) #wkb_geometry

09:57:44


In [141]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Reorder columns so population sum is last
gdf_reordered_columns = gdf[[c for c in gdf if c not in ['pop_eq']] + ['pop_eq']]
stats = gdf_reordered_columns

09:57:44


In [142]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Set a new ID column
stats = stats.reset_index(drop=True)
stats = gdf.drop('id', axis=1)
stats.insert(0, 'ID', range(len(gdf)))

09:57:44


### 2.2 Extract building count for each polygon using spatial join and count

In [143]:
#def drop_building_column(df):
#    if 'build' in df.columns:
#        df.drop('build', axis=1, inplace=True)
#    return df

In [144]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Join building centroids to admin polygons
print('    Joining points to polygons   Start: {}'.format(datetime.now().strftime("%H:%M:%S")))

points = build
polygons = stats

#drop_building_column(polygons)

# Joining the polygon attributes to each point
# Creates a point layer of all buildings with the attributes copied from the interesecting polygon uniquely for each point
joined_df = gpd.sjoin(
    points,
    polygons,
    how='inner',
    predicate='intersects')

print('    Joining points to polygons   Finish: {}'.format(datetime.now().strftime("%H:%M:%S")))

09:57:44
    Joining points to polygons   Start: 09:57:44
    Joining points to polygons   Finish: 09:57:45


In [145]:
# Count number of buildings within admin polygons (i.e. group by adm code)
print('    Counting number of buildings          Start: {}'.format(datetime.now().strftime("%H:%M:%S")))

build_count = joined_df.groupby(
    ['ID'],
    as_index=False,
)['geom'].count() # column is arbitrary

# Change column name to build_count
build_count.rename(columns = {'geom': 'bld_eq'}, inplace = True)

print('    Counting number of buildings          End:   {}'.format(datetime.now().strftime("%H:%M:%S")))

    Counting number of buildings          Start: 09:57:45
    Counting number of buildings          End:   09:57:45


In [146]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Merge build count back on to admin dataset
polygons = polygons.merge(
    build_count, 
    on=['ID'], 
    how='left')

09:57:45


### 2.3 Extract area for each polygon

In [147]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
polygons['km2_eq'] = polygons['geometry'].area.div(1000000)

09:57:45


### 3.1 Summarize by both province and earthquake intensity using pandas pivot table

In [148]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# pivot table to sum population by admin and intensity
pivoted = pd.pivot_table(polygons, values=['pop_eq','bld_eq', 'km2_eq'], index=pcode, columns='intensity', aggfunc='sum')

# fill NaN values with 0
pivoted.fillna(0, inplace=True)

# reset index to make admin a column again
pivoted = pivoted.reset_index()

09:57:45


### 4.1 Join pivot table columns to original province polygons

In [149]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
eq_stats = adm.merge(
            pivoted, 
            on=pcode, 
            how='left')

09:57:45


c:\Users\VMO\anaconda3\envs\earth2\lib\site-packages\geopandas\geodataframe.py:1470: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  result = DataFrame.merge(self, *args, **kwargs)
c:\Users\VMO\anaconda3\envs\earth2\lib\site-packages\geopandas\geodataframe.py:1470: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  result = DataFrame.merge(self, *args, **kwargs)


In [150]:
eq_stats.rename(columns=lambda x: x[0] + '_' + str(x[1]) if isinstance(x, tuple) and isinstance(x[1], int) else x, inplace=True)

In [151]:
out1_eq_stats = eq_stats

In [152]:
# Check which is the first pop_eq column that exists in the dataset

def get_first_column_startswith(df, prefix):
    for column in df.columns:
        if column.startswith(prefix):
            return column
    return None

prefix_pop = 'pop_eq'
prefix_build = 'bld_eq'
prefix_km2 = 'km2_eq'

first_column_pop = get_first_column_startswith(eq_stats, prefix_pop)
first_column_build = get_first_column_startswith(eq_stats, prefix_build)
first_column_km2 = get_first_column_startswith(eq_stats, prefix_km2)

#print(f"First column starting with '{prefix}': {first_column_pop}")

In [153]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Convert pop figures to int
eq_stats.loc[:,first_column_pop:'pop_eq_10'] = eq_stats.loc[:,first_column_pop:'pop_eq_10'].astype(int)
print('Converted population figures')

# Convert building figures to int
eq_stats.loc[:,first_column_build:'bld_eq_10'] = eq_stats.loc[:,first_column_build:'bld_eq_10'].astype(int)
print('Converted building figures')

# Round area figures to 2 decimal places
eq_stats.loc[:,first_column_km2:'km2_eq_10'] = eq_stats.loc[:,first_column_km2:'km2_eq_10'].round(1)
print('Rounded area figures')

09:57:46
Converted population figures
Converted building figures
Rounded area figures


In [154]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Reprojet back to 4326
eq_stats = eq_stats.to_crs('epsg:4326')

09:57:46


In [155]:
print('{}'.format(datetime.now().strftime("%H:%M:%S")))
# Save output to database
eq_stats.to_postgis('afg_admbnda_eq_hzda_stats_v12', con, if_exists='replace')

09:57:46


### 5.1 QA

In [156]:
#adm1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v06', con)
#adm2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v07', con)
#region = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_eq_hzda_stats_v08', con)

In [157]:
adm = adm1
print('build_count total', adm.build.sum())
print('build_count eq', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum total', adm.pop.sum())
print('pop_sum eq', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area total', adm.km2.sum())
print('area eq', adm.loc[:,['km2_eq_5', 'km2_eq_6', 'km2_eq_7', 'km2_eq_8', 'km2_eq_9', 'km2_eq_10']].sum().sum())

NameError: name 'adm1' is not defined

In [ ]:
adm = adm2
print('build_count', adm.build_ls_4.sum())
print('build_count all', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum', adm.pop_sum.sum())
print('pop_sum - excluding 0', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area_km2', adm.km2_ls_4.sum())

In [ ]:
adm = region
print('build_count', adm.build_ls_4.sum())
print('build_count all', adm.loc[:, ['bld_eq_5', 'bld_eq_6', 'bld_eq_7', 'bld_eq_8', 'bld_eq_9', 'bld_eq_10']].sum().sum())
print('pop_sum', adm.pop_sum.sum())
print('pop_sum - excluding 0', adm.loc[:,['pop_eq_5', 'pop_eq_6', 'pop_eq_7', 'pop_eq_8', 'pop_eq_9', 'pop_eq_10']].sum().sum())
print('area_km2', adm.km2_ls_4.sum())